In [56]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
np.random.seed(60)
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from collections import Counter
import keras
from keras.layers import Conv2D,Dense,Flatten
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Input
from keras.models import Model

In [74]:
def relative_cost(truth,pred):
    return np.mean(abs(pred-truth)/(.01+truth))




def relative_metric(truth,pred):
    return K.mean(abs(pred-truth)/(.01+truth))



def mse(truth,pred):
    return np.mean(1000*np.sqrt((truth-pred)**2))

In [58]:
def network():
    
    input_layer = Input(shape=(4,7,1),name="Matrix_Monthly_input")
    
    op1 = Conv2D(10,kernel_size=(4,1), strides=(1,1),name="Filter_4X1")(input_layer)
    
    op1 = Flatten()(op1)
    
    
    
    op2 = Conv2D(10,kernel_size=(1,7), strides=(1,1),name="Filter_1X7")(input_layer)
    
    op2 = Flatten()(op2)
    
    
    op3 = Conv2D(10,kernel_size=(1,2),strides=(1,1),name="Filter_1X2")(input_layer)
    
    op3 = Flatten()(op3)
    
    op4 = Conv2D(10,kernel_size=(2,1),strides=(1,1),name="Filter_2X1")(input_layer)
    
    op4 = Flatten()(op4)
    
    merge_layer = keras.layers.concatenate([op1, op2, op3,op4],name="Concatenation_1")

    
    merge_layer =  keras.layers.Dropout(.1)(merge_layer)
    
    
    
    
    final_layer = Dense(168*4,name='Final_Dense_Layer')(merge_layer)
    
    
    final_layer = keras.layers.LeakyReLU(alpha=0.01)(final_layer)
    
    #predictions = Dense(1, activation='sigmoid')(merge_layer)

    model = Model(inputs=input_layer, outputs=final_layer)
    
    return model
    
    

In [59]:
keys = pd.HDFStore('../Combined Datasets/daily.h5','r').keys()
print (keys[1])
np.random.shuffle(keys)
print (keys[1])

/1037
/3721


In [60]:
def return_data(keys,days,hours,n_months):

    appliance1 = 'use'
    appliance2 = 'use'

    
    X = np.array([days[i][appliance1].values for i in keys if (appliance1 in days[i].columns and appliance2 in days[i].columns )])[:,:28*n_months]/1000.0
    y = np.array([hours[i][appliance2].values for i in keys if (appliance1 in hours[i].columns and appliance2 in hours[i].columns )])[:,:(28*24*n_months)]/1000.0
    
    
    print (X.shape,y.shape)
    
    
    X = X.reshape((len(X)*n_months,28))
    y = y.reshape((len(y)*n_months,28*24))
        
    
    X = X.reshape((len(X)*n_months,4,7))
    y = y.reshape((len(y)*n_months,4,168))
    y = y.reshape((-1,168*4))
    X = np.expand_dims(X,axis=3)



    
    
    return X,y

In [85]:
train_keys = keys[:40]
test_keys = keys[-20:]
val_keys = keys[-40:-20]

In [62]:
days = pd.HDFStore('../Custom Datasets/Daily-2014-5.h5','r')
hours = pd.HDFStore('../Custom Datasets/Hourly-2014-5.h5','r')

In [63]:
train_may_x,train_may_y = return_data(train_keys,days,hours,1)
v_may_x,v_may_y = return_data(val_keys,days,hours,1)
test_x,test_y = return_data(test_keys,days,hours,1)

((40, 28), (40, 672))
((19, 28), (19, 672))
((19, 28), (19, 672))


In [86]:
len(keys[:40])

40

In [87]:
train_may_y[0].sum()

354.7877

In [88]:
#train_may_x

In [10]:
dir_name = "experiments"

In [89]:
model = network()
n_epochs = 1000
sgd = keras.optimizers.SGD(lr=0.01)
rmsprop  = keras.optimizers.RMSprop(lr=.001)
adagrad = keras.optimizers.adagrad(lr=.001)
model.compile(optimizer=sgd, loss='mean_squared_error',metrics=[relative_metric])
filepath="weights.best.h5"
checkpoint = ModelCheckpoint(dir_name+'/basic-{epoch:03d}.h5', verbose=5, monitor='val_loss',save_best_only=True, mode='auto')  
#model.fit(test_x,test_y)
model.fit(train_may_x, train_may_y, epochs=n_epochs, validation_data=[v_may_x,v_may_y], callbacks=[checkpoint], verbose=5,batch_size=3)

Train on 40 samples, validate on 19 samples
Epoch 1/1000

Epoch 00001: val_loss improved from inf to 1.00160, saving model to experiments/basic-001.h5
Epoch 2/1000

Epoch 00002: val_loss improved from 1.00160 to 0.99422, saving model to experiments/basic-002.h5
Epoch 3/1000

Epoch 00003: val_loss improved from 0.99422 to 0.98563, saving model to experiments/basic-003.h5
Epoch 4/1000

Epoch 00004: val_loss improved from 0.98563 to 0.97778, saving model to experiments/basic-004.h5
Epoch 5/1000

Epoch 00005: val_loss improved from 0.97778 to 0.97254, saving model to experiments/basic-005.h5
Epoch 6/1000

Epoch 00006: val_loss improved from 0.97254 to 0.96033, saving model to experiments/basic-006.h5
Epoch 7/1000

Epoch 00007: val_loss improved from 0.96033 to 0.95135, saving model to experiments/basic-007.h5
Epoch 8/1000

Epoch 00008: val_loss improved from 0.95135 to 0.94170, saving model to experiments/basic-008.h5
Epoch 9/1000

Epoch 00009: val_loss improved from 0.94170 to 0.93381, sa

In [93]:
all_files = os.listdir(dir_name)
all_files.sort()
weights_file = all_files[-1]

In [94]:
model.load_weights(dir_name+"/"+weights_file)

In [95]:
pred_may_train = model.predict(train_may_x)
pred_may_v = model.predict(v_may_x)
pred_test = model.predict(test_x)

In [96]:
train_cost = relative_cost(train_may_y,pred_may_train)
v_cost = relative_cost(v_may_y,pred_may_v)
test_cost = relative_cost(test_y,pred_test)

In [97]:
train_cost,test_cost,v_cost

(0.88659775, 0.9492752, 0.94071573)

In [98]:
model.save_weights('initial.h5')

In [129]:
model.load_weights('initial.h5')

In [130]:
train_cost = mse(train_may_y,pred_may_train)
v_cost = mse(v_may_y,pred_may_v)
test_cost = mse(test_y,pred_test)

In [131]:
train_cost,test_cost,v_cost

(390.23923, 382.9386, 334.60776)

# Data from Same Month

In [64]:
train_keys = keys[0:80]
test_keys = keys[-20:]
val_keys = keys[-40:-20]

In [65]:
len(keys)

138

In [66]:
train_may_x,train_may_y = return_data(train_keys,days,hours,1)
v_may_x,v_may_y = return_data(val_keys,days,hours,1)
test_x,test_y = return_data(test_keys,days,hours,1)

((79, 28), (79, 672))
((19, 28), (19, 672))
((19, 28), (19, 672))


In [67]:
model = network()
#model.load_weights('initial.h5')
n_epochs = 1000
sgd = keras.optimizers.SGD(lr=0.01)
rmsprop  = keras.optimizers.RMSprop(lr=.001)
adagrad = keras.optimizers.adagrad(lr=.001)
model.compile(optimizer=sgd, loss='mean_squared_error',metrics=[relative_metric])
filepath="weights.best.h5"
checkpoint = ModelCheckpoint(dir_name+'/basic-{epoch:03d}.h5', verbose=5, monitor='val_loss',save_best_only=True, mode='auto')  
#model.fit(test_x,test_y)
model.fit(train_may_x, train_may_y, epochs=n_epochs, validation_data=[v_may_x,v_may_y], callbacks=[checkpoint], verbose=5,batch_size=3)

Train on 79 samples, validate on 19 samples
Epoch 1/1000

Epoch 00001: val_loss improved from inf to 1.01972, saving model to experiments/basic-001.h5
Epoch 2/1000

Epoch 00002: val_loss improved from 1.01972 to 1.01479, saving model to experiments/basic-002.h5
Epoch 3/1000

Epoch 00003: val_loss improved from 1.01479 to 1.00829, saving model to experiments/basic-003.h5
Epoch 4/1000

Epoch 00004: val_loss improved from 1.00829 to 1.00123, saving model to experiments/basic-004.h5
Epoch 5/1000

Epoch 00005: val_loss improved from 1.00123 to 0.99656, saving model to experiments/basic-005.h5
Epoch 6/1000

Epoch 00006: val_loss improved from 0.99656 to 0.99349, saving model to experiments/basic-006.h5
Epoch 7/1000

Epoch 00007: val_loss improved from 0.99349 to 0.98579, saving model to experiments/basic-007.h5
Epoch 8/1000

Epoch 00008: val_loss improved from 0.98579 to 0.97566, saving model to experiments/basic-008.h5
Epoch 9/1000

Epoch 00009: val_loss improved from 0.97566 to 0.96542, sa

KeyboardInterrupt: 

In [68]:
all_files = os.listdir(dir_name)
all_files.sort()
weights_file = all_files[-1]

In [69]:
model.load_weights(dir_name+"/"+weights_file)

In [70]:
pred_may_train = model.predict(train_may_x)
pred_may_v = model.predict(v_may_x)
pred_test = model.predict(test_x)

In [75]:
train_cost = relative_cost(train_may_y,pred_may_train)
v_cost = relative_cost(v_may_y,pred_may_v)
test_cost = relative_cost(test_y,pred_test)

In [76]:
train_cost,test_cost,v_cost

(0.90954316, 0.8579047, 0.8801118)

In [77]:
train_cost = mse(train_may_y,pred_may_train)
v_cost = mse(v_may_y,pred_may_v)
test_cost = mse(test_y,pred_test)

In [78]:
train_cost,test_cost,v_cost

(352.12393, 395.45157, 348.8226)

# Data from Another Month


In [86]:
train_may_x,train_may_y = return_data(train_keys,days,hours,1)
v_may_x,v_may_y = return_data(val_keys,days,hours,1)

((40, 28), (40, 672))
((19, 28), (19, 672))


In [87]:
days = pd.HDFStore('../Custom Datasets/Daily-2014-6.h5','r')
hours = pd.HDFStore('../Custom Datasets/Hourly-2014-6.h5','r')

In [88]:
train_keys = keys[:40]
test_keys = keys[-20:]
val_keys = keys[-40:-20]

In [89]:
train_june_x,train_june_y = return_data(train_keys,days,hours,1)
v_june_x,v_june_y = return_data(val_keys,days,hours,1)

((40, 28), (40, 672))
((19, 28), (19, 672))


In [90]:
train_x = np.concatenate((train_may_x,train_june_x))
train_y = np.concatenate((train_may_y,train_june_y))
v_x = v_may_x
v_y = v_may_y

In [91]:
train_x.shape

(80, 4, 7, 1)

In [93]:
model = network()

n_epochs = 1000
sgd = keras.optimizers.SGD(lr=0.01)
rmsprop  = keras.optimizers.RMSprop(lr=.001)
adagrad = keras.optimizers.adagrad(lr=.001)
model.compile(optimizer=sgd, loss='mean_squared_error',metrics=[relative_metric])
filepath="weights.best.h5"
checkpoint = ModelCheckpoint(dir_name+'/basic-{epoch:03d}.h5', verbose=5, monitor='val_loss',save_best_only=True, mode='auto')  
#model.fit(test_x,test_y)
model.fit(train_x, train_y, epochs=n_epochs, validation_data=[v_x,v_y], callbacks=[checkpoint], verbose=5)

Train on 80 samples, validate on 19 samples
Epoch 1/1000

Epoch 00001: val_loss improved from inf to 1.59637, saving model to experiments/basic-001.h5
Epoch 2/1000

Epoch 00002: val_loss improved from 1.59637 to 1.57734, saving model to experiments/basic-002.h5
Epoch 3/1000

Epoch 00003: val_loss improved from 1.57734 to 1.56115, saving model to experiments/basic-003.h5
Epoch 4/1000

Epoch 00004: val_loss improved from 1.56115 to 1.55394, saving model to experiments/basic-004.h5
Epoch 5/1000

Epoch 00005: val_loss improved from 1.55394 to 1.54692, saving model to experiments/basic-005.h5
Epoch 6/1000

Epoch 00006: val_loss improved from 1.54692 to 1.54169, saving model to experiments/basic-006.h5
Epoch 7/1000

Epoch 00007: val_loss improved from 1.54169 to 1.53784, saving model to experiments/basic-007.h5
Epoch 8/1000

Epoch 00008: val_loss improved from 1.53784 to 1.53578, saving model to experiments/basic-008.h5
Epoch 9/1000

Epoch 00009: val_loss improved from 1.53578 to 1.52821, sa

In [94]:
all_files = os.listdir(dir_name)
all_files.sort()
weights_file = all_files[-1]

In [95]:
model.load_weights(dir_name+"/"+weights_file)

In [96]:
pred_train = model.predict(train_x)
pred_v = model.predict(v_x)
pred_test = model.predict(test_x)

In [97]:
train_cost = relative_cost(train_y,pred_train)
v_cost = relative_cost(v_y,pred_v)
test_cost = relative_cost(test_y,pred_test)

In [98]:
train_cost,test_cost,v_cost

(0.8024378, 0.83997643, 0.7953404)

In [101]:
train_cost = mse(train_y,pred_train)
v_cost = mse(v_y,pred_v)
test_cost = mse(test_y,pred_test)

In [102]:
train_cost,test_cost,v_cost

(406.76855, 419.935, 431.14896)

In [5]:
from keras.utils import plot_model
plot_model(model, to_file='experiment.png')

NameError: name 'model' is not defined